<a href="https://colab.research.google.com/github/thelalasis/mystudy/blob/master/pytroch_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytroch课程学习

In [0]:
import numpy as np
import torch

In [0]:
# create tensors
a = torch.tensor(3.)
b = torch.tensor(4.,requires_grad=True)
x = torch.tensor(5.,requires_grad=True)

In [4]:
y = a*b+x
print (y)
print (y.backward())

tensor(17., grad_fn=<AddBackward0>)
None


## 一个机器学习的入门样例
![替代文字](https://pic4.zhimg.com/80/v2-ad22857fb005da5418f0493a8971b8d3_hd.jpg)

yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1

yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [0]:
# Input (temp, rainfall, humidity), 这里的数据就是上表中的三个影响因子的数据
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges), 这里的数据就是上表中的数据结果数据. 也就是所谓的训练样本
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [6]:
# Convert inputs and targets to tensors, 这里将np的数据转为了pytorch的数据, 为了后续能用np读取数据后, 用torch再获取
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print (inputs)
print (targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [7]:
# Weights and biases 初始化参数, ??可以初始化为0么?
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[0.9644, 1.2468, 1.0221],
        [0.0723, 0.7857, 0.4764]], requires_grad=True)
tensor([ 0.3406, -1.5034], requires_grad=True)


In [0]:
# 定义计算模型, 返回
def model(x):
    return x @ w.t() + b

In [9]:
test = model(inputs)
print (test)
print (targets)

tensor([[198.2290,  76.8974],
        [263.2357, 104.7017],
        [310.6006, 137.6958],
        [190.1389,  57.2783],
        [258.1269, 112.2557]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [0]:
# MSE loss 定义计算偏差的均方差的方法
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Compute loss  计算偏差
loss = mse(test, targets)
print(loss)

tensor(14302.4785, grad_fn=<DivBackward0>)


In [13]:
# Compute gradients 计算梯度? 梯度的意义和作用
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

AttributeError: ignored

In [0]:
preds = model(inputs)
print (preds)
print (targets)

In [0]:
# Compute loss  计算偏差
loss = mse(preds, targets)
print(loss)

In [0]:
# Compute gradients 计算梯度, 梯度用来表明随着参数的变化, 目标值的变化方向
loss.backward()
print(w.grad)
print(b.grad)

In [0]:
# Adjust weights & reset gradients 基于梯度, 给出步进的步幅和方向, -=为递减方向, 步幅为1e-5, 重新评估的值赋予w和b
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [0]:
# 根据上一步, 得到的结果重新评估preds和targets之间的差异(loss)
print (w)
print (b)
preds = model(inputs)
print (preds)
print (targets)
loss = mse(preds, targets)
print(loss)

In [0]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

preds = model(inputs)
print (preds)
print (targets)
loss = mse(preds, targets)
print(loss)

## 在pytorch中实现
上文所述的内容, 都是利用数据计算的一般功能来实现的, pytorch本身具备了相关的计算方法设计, 可以直接利用. 

下面就通过pytorch来实现计算过程

In [0]:
import torch.nn as nn

In [0]:
# 输入数据样本, 比上文的样本量要大不少, 一共有15条
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [0]:
from torch.utils.data import TensorDataset

In [18]:
# Define dataset 采用了TensorDataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

TensorDataset 让我们可以使用数组索引表示法（上面代码中的 [0:3]）读取一小部分训练数据。它会返回一个元组（或配对），其中第一个元素包含所选行的输入变量，第二个元素包含目标

??[0:3]不是应该获取4行内容么?

In [19]:
train_ds[1:4]

(tensor([[ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]), tensor([[ 81., 101.],
         [119., 133.],
         [ 22.,  37.]]))

In [0]:
from torch.utils.data import DataLoader

In [0]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

这里分批获取数据, `batch_size`  用来描述采样的数量.

`DataLoader` 的具体方法需要进一步明确

In [22]:
# Define model 引入nn中的线性模型
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3190,  0.1300, -0.5139],
        [-0.3740,  0.1591, -0.5639]], requires_grad=True)
Parameter containing:
tensor([-0.5577, -0.1804], requires_grad=True)


In [23]:
# Parameters 这里获得的参数与上文中的model.weight等参数一致, 是获取全部参数的方法
list(model.parameters())

[Parameter containing:
 tensor([[-0.3190,  0.1300, -0.5139],
         [-0.3740,  0.1591, -0.5639]], requires_grad=True),
 Parameter containing:
 tensor([-0.5577, -0.1804], requires_grad=True)]

In [24]:
# Generate predictions 调用模型计算, 这里是pytorch定义的模型
preds = model(inputs)
preds

tensor([[-37.2344, -41.0747],
        [-51.0387, -56.3085],
        [-40.6992, -44.1121],
        [-46.5230, -52.3560],
        [-46.0633, -50.1904],
        [-37.2344, -41.0747],
        [-51.0387, -56.3085],
        [-40.6992, -44.1121],
        [-46.5230, -52.3560],
        [-46.0633, -50.1904],
        [-37.2344, -41.0747],
        [-51.0387, -56.3085],
        [-40.6992, -44.1121],
        [-46.5230, -52.3560],
        [-46.0633, -50.1904]], grad_fn=<AddmmBackward>)

In [0]:
# Import nn.functional
import torch.nn.functional as F

In [26]:
# Define loss function
loss_fn = F.mse_loss
# 调用loss函数计算损失值
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(18360.8125, grad_fn=<MseLossBackward>)


In [0]:
# Define optimizer 引入优化器 optim.SGD对model的参数进行优化 , 步幅为lr
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [0]:
# Utility function to train the model mun_epochs为步数
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [29]:
# 计算100步
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 222.4156
Epoch [20/100], Loss: 256.3550
Epoch [30/100], Loss: 84.0737
Epoch [40/100], Loss: 120.3417
Epoch [50/100], Loss: 123.8519
Epoch [60/100], Loss: 34.4435
Epoch [70/100], Loss: 61.2038
Epoch [80/100], Loss: 100.4000
Epoch [90/100], Loss: 93.9368
Epoch [100/100], Loss: 80.1817


In [30]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1861,  1.0194,  0.0996],
         [-0.0668,  1.0447,  0.1561]], requires_grad=True),
 Parameter containing:
 tensor([-0.5529, -0.1736], requires_grad=True)]